<a href="https://colab.research.google.com/github/86HenriqueSilva/_Analise_loteria_federal/blob/main/An%C3%A1lise_de_atrasos__Laborat%C3%B3rio_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


 **Edson Henrique**


 Análise e Desenvolvimento de Sistemas - UNINABUCO

Links:

[Linkedin](https://linkedin.com/in/henriquesilvatech) | [GitHub](https://github.com/86HenriqueSilva) | [Perfil na DIO](https://dio.me/users/edson_henrique)

Descrição: Profissional em constante aprimoramento técnico e acadêmico na área de Tecnologia da Informação e Desenvolvimento de Sistemas. Experiência consolidada em diversas tecnologias e metodologias, incluindo Python, Java, AWS, Docker, e Análise de Dados. Capacitado em liderança, gestão de projetos e implementações de soluções complexas em ambientes de TI.

Busco continuamente integrar novos conhecimentos e habilidades para enfrentar desafios emergentes no mercado digital e de nuvem.

Detalhes adicionais:

       Habilidades e Interesses:

    Proficiência em linguagens como Python, Java e Kotlin para desenvolvimento backend e análise de dados.
    Interesse crescente em inteligência artificial, machine learning e suas aplicações práticas.
    Familiaridade com metodologias ágeis, DevOps e práticas de desenvolvimento moderno.




# **Manual de Instruções - Análise de Loteria**

**Laboratório de Testes**

Objetivo:

Este programa permite analisar resultados de concursos da loteria federal , fornecendo informações sobre atrasos de números tais com a contagem da linha dos 10 ultimos concursos buscando o seu atraso com base na quantidade de concursos informados e frequência de aparições no total de concursos com base no curso digitado e a frequencia entre os ultimos 50 concursos.
   Lembrando que sua maior caracteristica e a analise de numeros de 0 a 9 em sua posição especifica de casa decimal  "M" "C"  "D"  "U" .

Instruções de Uso:

    Acesso aos Dados:
        Ao iniciar o programa, ele irá baixar automaticamente os dados mais recentes da loteria para análise.

    Seleção do Concurso:
        Digite o número do concurso que deseja analisar. Recomenda-se escolher um número igual ou superior a 50 para o valor do concurso, valores inferiores retornaram resultados inconsistentes por conta das métricas adotadas como base.

    Escolha do Prêmio:
        Selecione qual prêmio deseja analisar:
            1 - 1º Prêmio
            2 - 2º Prêmio
            3 - 3º Prêmio
            4 - 4º Prêmio
            5 - 5º Prêmio
            6 - Analisar todos os prêmios

    Quantidade de Concursos:
        Escolha quantos concursos deseja analisar, entre 5 e 15.

    Visualização dos Resultados:
        O programa exibirá os atrasos dos números não sorteados e a frequência de aparições de cada número em sua casa decimal.

        -linha dos 10 ultimos concursos
        -Contagem Geral
        -Linha dos 50 ultimos concursos

Observação Importante:

    Atenção: Valores de concursos inferiores a 50 podem resultar em análises inconsistentes.
     Recomenda-se escolher um número de concurso igual ou superior a 50 para resultados precisos.

Iniciar o Programa:

Para iniciar o programa, execute todas as células de código. Em seguida, siga as instruções fornecidas pelo programa para analisar os resultados desejados.

In [ ]:
import pandas as pd
import gdown
from tabulate import tabulate
from IPython.display import display, HTML

# Função para carregar os dados da planilha do Google Drive
def carregar_dados(planilha_url):
    """
    Função para baixar e carregar os dados de uma planilha do Google Drive.

    Args:
    - planilha_url (str): URL da planilha no Google Drive.

    Returns:
    - pd.DataFrame: DataFrame contendo os dados da planilha.
    """
    try:
        gdown.download(planilha_url, 'dados.xlsx', quiet=False)
        dados = pd.read_excel('dados.xlsx', engine='openpyxl')
        return dados
    except Exception as e:
        print(f"Erro ao carregar os dados da planilha: {e}")
        return None

# Função para analisar os números ausentes e calcular atrasos para um prêmio específico
def analisar_premio(dados, premio, num_concursos, concurso_escolhido):
    """
    Função para analisar os números ausentes e calcular os atrasos para um prêmio específico.

    Args:
    - dados (pd.DataFrame): DataFrame contendo os dados da loteria.
    - premio (int): Número do prêmio a ser analisado (1 a 5).
    - num_concursos (int): Número de concursos a serem analisados.
    - concurso_escolhido (int): Número do concurso específico a ser analisado.

    Returns:
    - dict: Dicionário com os resultados dos atrasos para cada coluna de prêmio.
    """
    premio_str = f"{premio}º"
    colunas_premio = [f"{letra} {premio_str}" for letra in ['M', 'C', 'D', 'U']]

    if not all(coluna in dados.columns for coluna in colunas_premio):
        print(f"Prêmio {premio_str} não encontrado nos dados.")
        return None

    # Filtrar os dados até o limite do concurso escolhido
    dados_filtrados = dados[dados['Concurso'] <= concurso_escolhido]

    indice_concurso = dados_filtrados[dados_filtrados['Concurso'] == concurso_escolhido].index
    if len(indice_concurso) == 0:
        print(f"Concurso {concurso_escolhido} não encontrado nos dados.")
        return None

    indice_concurso = indice_concurso[0]
    ultimos_concursos = dados_filtrados.iloc[max(0, indice_concurso - num_concursos + 1):indice_concurso + 1].iloc[::-1]
    resultados = {}

    for coluna in colunas_premio:
        numeros_presentes = set(ultimos_concursos[coluna].astype(str))
        numeros_ausentes = set([str(i) for i in range(10)]) - numeros_presentes

        atrasos = []
        for num in sorted(numeros_ausentes):
            indices_ausencia = dados_filtrados[dados_filtrados[coluna].astype(str).apply(lambda x: num in x)].index.tolist()
            if indices_ausencia:
                atraso = concurso_escolhido - max(indices_ausencia)
                atrasos.append((num, max(indices_ausencia), atraso))
            else:
                atrasos.append((num, -1, len(dados_filtrados)))

        atrasos.sort(key=lambda x: x[2], reverse=True)
        resultados[coluna] = atrasos

    return resultados

# Função para calcular a frequência dos números de 0 a 9 para um prêmio específico
def calcular_frequencia_numeros(dados, premio, concurso_escolhido):
    """
    Função para calcular a frequência dos números de 0 a 9 para um prêmio específico.

    Args:
    - dados (pd.DataFrame): DataFrame contendo os dados da loteria.
    - premio (int): Número do prêmio a ser analisado (1 a 5).
    - concurso_escolhido (int): Número do concurso específico a ser analisado.

    Returns:
    - dict: Dicionário com a frequência dos números para cada coluna de prêmio.
    """
    premio_str = f"{premio}º"
    colunas_premio = [f"{letra} {premio_str}" for letra in ['M', 'C', 'D', 'U']]

    # Filtrar os dados até o limite do concurso escolhido
    dados_filtrados = dados[dados['Concurso'] <= concurso_escolhido]

    frequencia = {coluna: dados_filtrados[coluna].astype(str).value_counts().sort_index() for coluna in colunas_premio}
    for coluna in frequencia:
        frequencia[coluna] = frequencia[coluna][frequencia[coluna] != 0]  # Remover contagens zero
    return frequencia

# Função para calcular a frequência dos números de 0 a 9 nos últimos 50 concursos para um prêmio específico
def calcular_frequencia_ultimos_50(dados, premio, concurso_escolhido):
    """
    Função para calcular a frequência dos números de 0 a 9 nos últimos 50 concursos para um prêmio específico.

    Args:
    - dados (pd.DataFrame): DataFrame contendo os dados da loteria.
    - premio (int): Número do prêmio a ser analisado (1 a 5).
    - concurso_escolhido (int): Número do concurso específico a ser analisado.

    Returns:
    - dict: Dicionário com a frequência dos números nos últimos 50 concursos para cada coluna de prêmio.
    """
    premio_str = f"{premio}º"
    colunas_premio = [f"{letra} {premio_str}" for letra in ['M', 'C', 'D', 'U']]

    # Filtrar os dados até o limite do concurso escolhido
    dados_filtrados = dados[dados['Concurso'] <= concurso_escolhido]

    indice_concurso = dados_filtrados[dados_filtrados['Concurso'] == concurso_escolhido].index
    if len(indice_concurso) == 0:
        print(f"Concurso {concurso_escolhido} não encontrado nos dados.")
        return None

    indice_concurso = indice_concurso[0]
    ultimos_50 = dados_filtrados.iloc[max(0, indice_concurso - 49):indice_concurso + 1]

    frequencia_50 = {coluna: ultimos_50[coluna].astype(str).value_counts().sort_index() for coluna in colunas_premio}
    for coluna in frequencia_50:
        frequencia_50[coluna] = frequencia_50[coluna][frequencia_50[coluna] != 0]  # Remover contagens zero
    return frequencia_50

# Função para exibir os resultados de forma organizada com tabulação
def exibir_resultados(resultados_completos, frequencia_geral, frequencia_50):
    """
    Função para exibir os resultados de forma organizada com tabulação.

    Args:
    - resultados_completos (dict): Dicionário contendo os resultados completos dos atrasos para cada prêmio.
    - frequencia_geral (dict): Dicionário contendo a frequência geral dos números de 0 a 9 para cada prêmio.
    - frequencia_50 (dict): Dicionário contendo a frequência dos números nos últimos 50 concursos para cada prêmio.
    """
    for premio, resultados in resultados_completos.items():
        print(f"\nPrêmio {premio}º:\n")

        for coluna, atrasos in resultados.items():
            print(f"{coluna}:")
            print(tabulate(atrasos, headers=["Número", "Última aparição", "Atraso"]))
            print("")

    print("\nFrequência Geral dos Números de 0 a 9:")
    for premio, freq in frequencia_geral.items():
        print(f"\nPrêmio {premio}º:")
        df_geral = pd.DataFrame(freq).fillna(0).astype(int)
        df_geral.columns = [col.replace(' ', '') for col in df_geral.columns]
        print(tabulate(df_geral, headers="keys", tablefmt="pretty"))

    print("\nFrequência dos Números nos Últimos 50 Concursos:")
    for premio, freq in frequencia_50.items():
        print(f"\nPrêmio {premio}º:")
        df_50 = pd.DataFrame(freq).fillna(0).astype(int)
        df_50.columns = [col.replace(' ', '') for col in df_50.columns]
        print(tabulate(df_50, headers="keys", tablefmt="pretty"))

# Função para exibir o menu de seleção de prêmios
def exibir_menu_premios():
    """
    Função para exibir o menu de seleção de prêmios.
    """
    print("Escolha o prêmio que deseja analisar:")
    print("1 - 1º Prêmio")
    print("2 - 2º Prêmio")
    print("3 - 3º Prêmio")
    print("4 - 4º Prêmio")
    print("5 - 5º Prêmio")
    print("6 - Analisar todos os prêmios")
    print("0 - Sair")

# Função para obter o concurso específico que o usuário deseja analisar
def obter_concurso_especifico(dados):
    """
    Função para obter o número do concurso específico que o usuário deseja analisar.

    Args:
    - dados (pd.DataFrame): DataFrame contendo os dados da loteria.

    Returns:
    - int: Número do concurso escolhido pelo usuário.
    """
    while True:
        try:
            concurso_escolhido = int(input("Digite o número do concurso que deseja analisar: "))
            if concurso_escolhido < 50:
                display(HTML(f'<p style="color:red; font-weight:bold;">Atenção: Valores de concursos inferiores a 50 podem resultar em análises inconsistentes.</p>'))
            if concurso_escolhido not in dados['Concurso'].values:
                raise ValueError("Concurso não encontrado nos dados.")
            return concurso_escolhido
        except ValueError as e:
            print(f"Erro: {e}. Por favor, digite um número de concurso válido.")

# Função para obter o número de concursos específico que o usuário deseja analisar
def obter_numero_concursos():
    """
    Função para obter o número de concursos que o usuário deseja analisar (entre 5 e 15).

    Returns:
    - int: Número de concursos desejado pelo usuário.
    """
    while True:
        try:
            num_concursos = int(input("Digite quantos concursos deseja analisar (5 a 15): "))
            if num_concursos < 5 or num_concursos > 15:
                raise ValueError("Número de concursos fora do intervalo permitido.")
            return num_concursos
        except ValueError as e:
            print(f"Erro: {e}. Por favor, digite um número válido entre 5 e 15.")

# Menu principal interativo
def menu_principal():
    planilha_url = "https://drive.google.com/uc?export=download&id=10IcF31FnO4oWh2dYrq8qJWYR7Q8tTJkP"
    dados = carregar_dados(planilha_url)

    if dados is None:
        return

    print(f"O último concurso na planilha é: {dados['Concurso'].max()}")

    concurso_escolhido = obter_concurso_especifico(dados)

    while True:
        exibir_menu_premios()
        opcao = input("Digite o número da opção desejada: ")

        if opcao == '0':
            print("Saindo do programa.")
            break

        if opcao not in ['1', '2', '3', '4', '5', '6']:
            print("Opção inválida. Por favor, escolha uma opção válida.")
            continue

        premio = int(opcao)
        num_concursos = obter_numero_concursos()

        resultados_completos = {}
        frequencia_geral = {}
        frequencia_50 = {}

        if opcao == '6':
            for premio in range(1, 6):
                resultados = analisar_premio(dados, premio, num_concursos, concurso_escolhido)
                if resultados:
                    resultados_completos[premio] = resultados
                    frequencia_geral[premio] = calcular_frequencia_numeros(dados, premio, concurso_escolhido)
                    frequencia_50[premio] = calcular_frequencia_ultimos_50(dados, premio, concurso_escolhido)
        else:
            resultados = analisar_premio(dados, premio, num_concursos, concurso_escolhido)
            if resultados:
                resultados_completos[premio] = resultados
                frequencia_geral[premio] = calcular_frequencia_numeros(dados, premio, concurso_escolhido)
                frequencia_50[premio] = calcular_frequencia_ultimos_50(dados, premio, concurso_escolhido)

        exibir_resultados(resultados_completos, frequencia_geral, frequencia_50)

# Executa o menu principal se o script for executado diretamente
if __name__ == "__main__":
    menu_principal()

Downloading...
From: https://drive.google.com/uc?export=download&id=10IcF31FnO4oWh2dYrq8qJWYR7Q8tTJkP
To: /content/dados.xlsx
100%|██████████| 509k/509k [00:00<00:00, 59.8MB/s]


O último concurso na planilha é: 5882
Escolha o prêmio que deseja analisar:
1 - 1º Prêmio
2 - 2º Prêmio
3 - 3º Prêmio
4 - 4º Prêmio
5 - 5º Prêmio
6 - Analisar todos os prêmios
0 - Sair

Prêmio 1º:

M 1º:
  Número    Última aparição    Atraso
--------  -----------------  --------
       9               5856        25
       0               5857        24

C 1º:
  Número    Última aparição    Atraso
--------  -----------------  --------
       0               5856        25
       6               5858        23

D 1º:
  Número    Última aparição    Atraso
--------  -----------------  --------
       9               5851        30
       1               5853        28
       7               5858        23

U 1º:
  Número    Última aparição    Atraso
--------  -----------------  --------
       7               5857        24


Frequência Geral dos Números de 0 a 9:

Prêmio 1º:
+---+-----+-----+-----+-----+
|   | M1º | C1º | D1º | U1º |
+---+-----+-----+-----+-----+
| 0 | 621 | 631 | 615 | 

KeyboardInterrupt: Interrupted by user

# Apresentação do Projeto


**Objetivo**

Meu projeto tem como objetivo analisar dados de resultados de uma loteria específica, permitindo aos usuários explorar e entender padrões de números premiados e suas frequências ao longo dos concursos.
Funcionalidades Principais

    Carregamento de Dados: Utilizei a biblioteca gdown para baixar e pandas para carregar dados de uma planilha do Google Drive contendo os resultados da loteria.

    Análise de Números Ausentes e Atrasos:
    Desenvolvi uma função analisar_premio que analisa os números ausentes e calcula os atrasos para um prêmio específico, considerando um número de concursos escolhido.

    Cálculo de Frequência de Números:
        Implementei as funções calcular_frequencia_numeros e calcular_frequencia_ultimos_50 para calcular a frequência dos números de 0 a 9 até um concurso escolhido e nos últimos 50 concursos, respectivamente.

    Apresentação de Resultados:
        Utilizei a biblioteca tabulate para formatar e apresentar os resultados de forma organizada em tabelas,
         incluindo a frequência geral e nos últimos 50 concursos para cada prêmio.

    Interface Interativa:
        Desenvolvi um menu interativo usando Python,
         onde os usuários podem escolher qual prêmio analisar,
          quantos concursos considerar e visualizar os resultados detalhados.

Bibliotecas Utilizadas

    pandas:
    Essencial para manipulação e análise de dados tabulares, permitindo carregar e processar os dados da loteria de maneira eficiente.

    gdown:
    Utilizada para baixar arquivos diretamente do Google Drive via URL, facilitando o acesso aos dados da planilha.

    tabulate:
     Utilizada para formatar os resultados em tabelas claras e organizadas,
      adequadas tanto para visualização direta quanto para relatórios profissionais.

    IPython.display:
     Utilizada para exibir mensagens formatadas em HTML,
      como alertas sobre escolhas potencialmente inconsistentes de concursos,
       garantindo uma experiência de usuário mais informativa.

**Valor como Portfólio**

Este projeto demonstra minhas competências em:

    Análise de Dados:
    Manipulação avançada de dados usando pandas para filtrar,
    calcular frequências e identificar padrões nos resultados da loteria.

    Desenvolvimento Python:
     Implementação de funções eficientes e estruturadas em Python para análise de dados
     em um contexto real.

    Visualização de Dados:
     Apresentação clara e organizada dos resultados utilizando tabelas formatadas,
     destacando minha capacidade de comunicar insights de dados de maneira eficaz.

**Conclusão**

Ao apresentar este projeto como parte do meu portfólio, destaco não apenas minhas habilidades técnicas, mas também minha capacidade de desenvolver soluções robustas e funcionais para análise de dados.

Este projeto representa meu compromisso com a excelência na área de tecnologia e análise de dados, e estou entusiasmado para explorar oportunidades que permitam aplicar essas habilidades em contextos desafiadores e inovadores.